In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mantra.simplicial import SimplicialDataset
import torch
import numpy as np
import torchvision.transforms as transforms
from mantra.transforms import OrientableToClassTransform, SimplicialComplexTransform
from mantra.dataloaders import SimplicialDataLoader
from topomodelx.utils.sparse import from_sparse

In [3]:
from mantra.transforms import SimplicialComplexDegreeTransform, SimplicialComplexEdgeCoadjacencyDegreeTransform, \
    SimplicialComplexEdgeAdjacencyDegreeTransform, SimplicialComplexTriangleCoadjacencyDegreeTransform, \
    OrientableToClassSimplicialComplexTransform, DimTwoHodgeLaplacianSimplicialComplexTransform, \
    DimOneHodgeLaplacianDownSimplicialComplexTransform, DimOneHodgeLaplacianUpSimplicialComplexTransform, \
    DimZeroHodgeLaplacianSimplicialComplexTransform

tr = transforms.Compose(
            [SimplicialComplexTransform(), 
             SimplicialComplexDegreeTransform(),
             SimplicialComplexEdgeCoadjacencyDegreeTransform(),
             SimplicialComplexEdgeAdjacencyDegreeTransform(),
             SimplicialComplexTriangleCoadjacencyDegreeTransform(),
             DimZeroHodgeLaplacianSimplicialComplexTransform(),
             DimOneHodgeLaplacianUpSimplicialComplexTransform(),
             DimOneHodgeLaplacianDownSimplicialComplexTransform(),
             DimTwoHodgeLaplacianSimplicialComplexTransform(),
             OrientableToClassSimplicialComplexTransform()]
        )

In [4]:
dataset = SimplicialDataset(root="./data", transform=tr)
dloader = SimplicialDataLoader(dataset, batch_size=3, shuffle=False)

In [6]:
# Take the first batch
batch, signals_belonging, len_batch = next(iter(dloader))